# 특정 장르 추출

In [ ]:
import os
import shutil

# 폴더 경로 설정
source_folder = r'G:\내 드라이브\project_poster\x_train - 복사본' 
# 이미지 파일이 있는 폴더
destination_folder = r'G:\내 드라이브\project_poster\comedy'
# 코미디 장르 이미지를 옮길 폴더

# 만약 대상 폴더가 없으면 새로 생성
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

# 코미디 장르 비트 (예시: 코미디가 3번째 비트에 해당된다고 가정)
# "코메디": 1 << 18
COMEDY_BIT_MASK = 1 << 18  # 3번째 비트를 확인

# 파일명을 읽어 코미디 장르를 포함하는 파일만 필터링
for filename in os.listdir(source_folder):
    if filename.endswith('.jpg'):
        try:
            # 파일명에서 장르 코드를 추출
            title, genre_code_str = filename.split('_')
            genre_code = int(genre_code_str.split('.')[0])  # 확장자 제거하고 장르 코드만 추출

            # 코미디 장르 비트가 켜져 있는지 확인
            if genre_code & COMEDY_BIT_MASK:
                # 코미디 장르가 포함된 파일일 경우 대상 폴더로 이동
                source_file_path = os.path.join(source_folder, filename)
                destination_file_path = os.path.join(destination_folder, filename)
                
                # 파일 이동
                shutil.move(source_file_path, destination_file_path)
                print(f'Moved: {filename}')
        except Exception as e:
            print(f"Error processing file {filename}: {e}")

# 데이터 증강

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, save_img
import random

# 이미지 폴더 경로
input_folder = r'G:\내 드라이브\project_poster\comedy'  # 원본 이미지가 저장된 폴더
output_folder = r'G:\내 드라이브\project_poster\comedy_augmented'  # 증강된 이미지를 저장할 폴더
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# 이미지 데이터 생성기 설정
datagen = ImageDataGenerator(
    rotation_range=40,          # 0~40도 회전
    width_shift_range=0.2,      # 20% 수평 이동
    height_shift_range=0.2,     # 20% 수직 이동
    shear_range=0.2,            # 전단 변형
    zoom_range=0.2,             # 20% 확대/축소
    horizontal_flip=True,        # 좌우 반전
    fill_mode='nearest'         # 새로 생성된 픽셀을 채우는 방법
)

# 사용하고자 하는 증강 방법 리스트
augmentation_methods = [
    "rotation",
    "width_shift",
    "height_shift",
    "shear",
    "zoom",
    "horizontal_flip",
    "add_noise",  # 노이즈 추가
    "crop",        # 크롭
    "scaling",     # 스케일링
]

# 폴더 내 이미지 파일을 불러와서 데이터 증강 적용
for filename in os.listdir(input_folder):
    img_path = os.path.join(input_folder, filename)
    img = load_img(img_path)  # 이미지 불러오기
    x = img_to_array(img)      # 이미지를 배열로 변환
    x = np.expand_dims(x, axis=0)  # 차원 확장

    # 랜덤으로 증강 방법 선택
    method = random.choice(augmentation_methods)

    if method == "rotation":
        for i, batch in enumerate(datagen.flow(x, batch_size=1)):
            save_img(os.path.join(output_folder, f'rotation!{i}!{filename}'), batch[0])
            if i >= 1:  # 각 방법당 1개만 생성
                break

    elif method == "width_shift":
        for i, batch in enumerate(datagen.flow(x, batch_size=1)):
            save_img(os.path.join(output_folder, f'widthshift!{i}!{filename}'), batch[0])
            if i >= 1:
                break

    elif method == "height_shift":
        for i, batch in enumerate(datagen.flow(x, batch_size=1)):
            save_img(os.path.join(output_folder, f'heightshift!{i}!{filename}'), batch[0])
            if i >= 1:
                break

    elif method == "shear":
        for i, batch in enumerate(datagen.flow(x, batch_size=1)):
            save_img(os.path.join(output_folder, f'shear!{i}!{filename}'), batch[0])
            if i >= 1:
                break

    elif method == "zoom":
        for i, batch in enumerate(datagen.flow(x, batch_size=1)):
            save_img(os.path.join(output_folder, f'zoom!{i}!{filename}'), batch[0])
            if i >= 1:
                break

    elif method == "horizontal_flip":
        for i, batch in enumerate(datagen.flow(x, batch_size=1)):
            save_img(os.path.join(output_folder, f'horizontalflip!{i}!{filename}'), batch[0])
            if i >= 1:
                break

    elif method == "add_noise":
        noise = np.random.normal(0, 0.1, x[0].shape)  # 노이즈 추가
        noisy_image = x[0] + noise
        noisy_image = np.clip(noisy_image, 0, 255).astype('uint8')
        save_img(os.path.join(output_folder, f'addnoise!{i}!{filename}'), noisy_image)

    elif method == "crop":
        # 랜덤 크롭 (예: 중앙 부분 70%)
        h, w, _ = x[0].shape
        crop_h, crop_w = int(h * 0.7), int(w * 0.5)
        x_center, y_center = h // 2, w // 2
        cropped_image = x[0][x_center - crop_h // 2: x_center + crop_h // 2,
                             y_center - crop_w // 2: y_center + crop_w // 2]
        save_img(os.path.join(output_folder, f'crop!{i}!{filename}'), cropped_image)

    elif method == "scaling":
        scaling_factor = random.uniform(0.5, 1.5)  # 랜덤 스케일
        scaled_image = np.clip(x[0] * scaling_factor, 0, 255).astype('uint8')
        save_img(os.path.join(output_folder, f'scaling!{i}!{filename}'), scaled_image)

print("이미지 증강 완료!")